In [1]:
import requests
from bs4 import BeautifulSoup
import json


In [2]:
from currency_converter import CurrencyConverter, SINGLE_DAY_ECB_URL
cc = CurrencyConverter(SINGLE_DAY_ECB_URL)

In [3]:
x, curr = 5000, 'SEK'

print(round(cc.convert(x, curr), 3), round(cc.convert(x, curr)))

454.422 454


In [14]:
x, curr = 1500, 'NOK'

print(round(cc.convert(x, curr), 3), round(cc.convert(x, curr)))

137.149 137


In [5]:
from_to = "Gdansk/Bodo"

In [6]:
base_url = "https://www.rome2rio.com/map/"
    
tmp_url = base_url + from_to  
s = requests.session()
r = s.get(tmp_url)
print(r.status_code, tmp_url)

200 https://www.rome2rio.com/map/Gdansk/Bodo


In [7]:
soup = BeautifulSoup(r.text, "html.parser")
#print(soup.prettify())

In [8]:
# finds common info about the trip
title = soup.find('meta', property="og:title")
print(title['content'])

Gdańsk to Bodø - 6 ways to travel via train, plane, bus, and night train


In [9]:
# gets quickest and cheapest ways
description = soup.find('meta', property="og:description")
print(description['content'])

The cheapest way to get from Gdańsk to Bodø costs only kr 1782, and the quickest way takes just 5¼ hours. Find the travel option that best suits you.


In [10]:
# finds nodes geolocation info
nodes = soup.find('meta', id="deeplinkNodes")
nodes['content']

'[["GeoCoderResultV1","Gdańsk","Gdańsk, Poland",54.35227,18.64912,"City","PL","Gdańsk","False",2.5],["GeoCoderResultV1","Bodø","Bodø, Norway",67.28,14.40501,"Town","NO","Bodø","False",0.5]]'

In [11]:
# find trip`s pathes
pathes = soup.find('meta', id="deeplinkTrip")
pathes = json.loads(pathes['content'])[2][1]

In [12]:
# calculates number of pathes
num_pathes = len([i for i in pathes])
print(f'{num_pathes = }')

num_pathes = 7


In [ ]:
# get all pathes wirh numbers
for i, item in enumerate(pathes):
    print('path ', i, '\t', item, '\n\n')

In [ ]:
# unpacking all routes from each path
for path_id, path in enumerate(pathes):
    for i, route in enumerate(path):
        print(path_id, '\t', i, '\t', route, '\n')

In [13]:
# unpacking each route's info from the section 8 (except last one [:-1], because it is 'hotel')
for path_id, path in enumerate(pathes):
    print(f'path # {path_id}\n')
    for i, route in enumerate(path[8]):
        for j, way in enumerate(route):
            print(path_id, '\t', i, '\t', j, '\t', way, '\n')

path # 0

0 	 0 	 0 	 car 

0 	 0 	 1 	 taxi 

0 	 0 	 2 	 yellow 

0 	 0 	 3 	 1012.533 

0 	 0 	 4 	 0 

0 	 0 	 5 	 16.87498 

0 	 0 	 6 	 ['node', 'Gdańsk', 54.35227, 18.64912, 'PL', 'Europe/Warsaw', 2.5] 

0 	 0 	 7 	 ['airport', 'Gdansk (GDN)', 54.38073, 18.46845] 

0 	 0 	 8 	  

0 	 0 	 9 	 False 

0 	 0 	 10 	  

0 	 0 	 11 	  

0 	 0 	 12 	 False 

0 	 0 	 13 	 [[120, 'NOK', 125.02, 'False', ''], [150, 'NOK', 147.75, 'False', ''], [150, 'NOK', 147.75, 'False', '']] 

0 	 0 	 14 	 [[50, 'PLN', 54.11, 'False', ''], [65, 'PLN', 63.94, 'False', ''], [65, 'PLN', 63.94, 'False', '']] 

0 	 0 	 15 	 53 

0 	 0 	 16 	 53 

0 	 0 	 17 	 [['Neptun Taxi', '+48 58 511 15 55', 'https://neptuntaxi.pl/platnosc-karta-w-taksowce-czy-to-mozliwe/', 'neptuntaxi.pl'], ['Hallo Taxi', '+48 58 301 59 59', 'https://hallotaxi.gda.pl/en/homepage/', 'hallotaxi.gda.pl'], ['Dajan Taxi', '+48 58 306 06 33', '', ''], ['Airport Taxi Gdańsk', '+48 734 134 059', 'https://airporttaxigdansk.com.pl/pl/', 'airport

In [ ]:
# unpacking the prices (min, mid, max) of the path from the section 20
for path_id, path in enumerate(pathes, start=1):
    print(path_id, '\n\t', path[20], '\n')

In [ ]:
# unpacking the prices (min, mid, max) for each route of the path from the section 20
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path[20]):
        print(path_id, '\n\t', i, '\t', route, '\n')